In [9]:
import polars as pl
import duckdb

airlines = pl.read_parquet('./parquet/airlines.parquet')
airports = pl.read_parquet('./parquet/airports.parquet')
flights = pl.read_parquet('./parquet/flights.parquet')
planes = pl.read_parquet('./parquet/planes.parquet')
weather = pl.read_parquet('./parquet/weather.parquet')

## add calculated new variables (mutate)

In [17]:
query = f'''
from flights
select
    *,
    gain: arr_delay - dep_delay,
    hours: air_time / 60,
    gain_per_hour: gain / hours -- note: created column can be used immediately
'''

con = duckdb.connect('./db/nycflights13.duckdb')
result = con.sql(query).df()
con.close()

display(result.head(4))

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,gain,hours,gain_per_hour
0,2013,6,30,940,15,1216,-4,VX,N626VA,407,JFK,LAX,313,2475,9,40,-19,5.216667,-3.642173
1,2013,5,7,1657,-3,2104,10,DL,N3760C,329,JFK,SJU,216,1598,16,57,13,3.600000,3.611111
2,2013,12,8,859,-1,1238,11,DL,N712TW,422,JFK,LAX,376,2475,8,59,12,6.266667,1.914894
3,2013,5,14,1841,-4,2122,-34,DL,N914DL,2391,JFK,TPA,135,1005,18,41,-30,2.250000,-13.333333


In [18]:
flights.with_columns(
    gain = pl.col('arr_delay') - pl.col('dep_delay'),
    hours = pl.col('air_time') / 60,
).with_columns(
    gain_per_hour = pl.col('gain') / pl.col('hours'),
).head(4)

year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,gain,hours,gain_per_hour
i64,i64,i64,i64,i64,i64,i64,str,str,i64,str,str,i64,i64,i64,i64,i64,f64,f64
2013,6,30,940,15,1216,-4,"""VX""","""N626VA""",407,"""JFK""","""LAX""",313,2475,9,40,-19,5.216667,-3.642173
2013,5,7,1657,-3,2104,10,"""DL""","""N3760C""",329,"""JFK""","""SJU""",216,1598,16,57,13,3.6,3.611111
2013,12,8,859,-1,1238,11,"""DL""","""N712TW""",422,"""JFK""","""LAX""",376,2475,8,59,12,6.266667,1.914894
2013,5,14,1841,-4,2122,-34,"""DL""","""N914DL""",2391,"""JFK""","""TPA""",135,1005,18,41,-30,2.25,-13.333333


## only calculated variables (transmute)

In [19]:
query = f'''
from flights
select
    gain: arr_delay - dep_delay,
    hours: air_time / 60,
    gain_per_hour: gain / hours -- note: created column can be used immediately
'''

con = duckdb.connect('./db/nycflights13.duckdb')
result = con.sql(query).df()
con.close()

display(result.head(4))

,gain,hours,gain_per_hour
0,-19,5.216667,-3.642173
1,13,3.600000,3.611111
2,12,6.266667,1.914894
3,-30,2.250000,-13.333333


In [20]:
flights.select(
    gain = pl.col('arr_delay') - pl.col('dep_delay'),
    hours = pl.col('air_time') / 60,
).with_columns(
    gain_per_hour = pl.col('gain') / pl.col('hours'),
).head(4)

gain,hours,gain_per_hour
i64,f64,f64
-19,5.216667,-3.642173
13,3.6,3.611111
12,6.266667,1.914894
-30,2.25,-13.333333


## sorting

In [25]:
query = f'''
from flights
order by arr_delay desc, arr_delay desc
limit 10
'''

con = duckdb.connect('./db/nycflights13.duckdb')
result = con.sql(query).df()
con.close()

display(result)

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
0,2013,1,9,641,1301,1242,1272,HA,N384HA,51,JFK,HNL,640,4983,6,41
1,2013,12,19,734,849,1046,847,DL,N375NC,1223,EWR,SLC,290,1969,7,34
2,2013,6,27,753,803,937,802,AA,N571AA,2019,LGA,STL,134,888,7,53
3,2013,6,27,615,790,853,769,DL,N372DA,503,JFK,SAN,312,2446,6,15
4,2013,6,28,121,502,329,490,DL,N360NB,2042,EWR,ATL,106,746,1,21
5,2013,9,2,2218,473,2349,444,EV,N744EV,4949,LGA,GSO,69,461,22,18
6,2013,4,18,2200,427,106,435,UA,N811UA,479,LGA,IAH,208,1416,22,0
7,2013,11,4,1822,413,2143,434,AA,N3GRAA,1139,LGA,DFW,210,1389,18,22
8,2013,4,24,2048,423,2207,422,WN,N931WN,2226,LGA,MDW,117,725,20,48
9,2013,7,10,2054,355,102,421,9E,N937XJ,3325,JFK,DFW,191,1391,20,54


In [26]:
flights.sort(['arr_delay', 'dep_delay'], descending=[True, True]).head(10)

year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
i64,i64,i64,i64,i64,i64,i64,str,str,i64,str,str,i64,i64,i64,i64
2013,1,9,641,1301,1242,1272,"""HA""","""N384HA""",51,"""JFK""","""HNL""",640,4983,6,41
2013,12,19,734,849,1046,847,"""DL""","""N375NC""",1223,"""EWR""","""SLC""",290,1969,7,34
2013,6,27,753,803,937,802,"""AA""","""N571AA""",2019,"""LGA""","""STL""",134,888,7,53
2013,6,27,615,790,853,769,"""DL""","""N372DA""",503,"""JFK""","""SAN""",312,2446,6,15
2013,6,28,121,502,329,490,"""DL""","""N360NB""",2042,"""EWR""","""ATL""",106,746,1,21
2013,9,2,2218,473,2349,444,"""EV""","""N744EV""",4949,"""LGA""","""GSO""",69,461,22,18
2013,4,18,2200,427,106,435,"""UA""","""N811UA""",479,"""LGA""","""IAH""",208,1416,22,0
2013,11,4,1822,413,2143,434,"""AA""","""N3GRAA""",1139,"""LGA""","""DFW""",210,1389,18,22
2013,4,24,2048,423,2207,422,"""WN""","""N931WN""",2226,"""LGA""","""MDW""",117,725,20,48


## summaries

In [31]:
query = f'''
from flights
select delay: mean(dep_delay)
'''

con = duckdb.connect('./db/nycflights13.duckdb')
result = con.sql(query).df()
con.close()

display(result)

,delay
0,12.705147


In [30]:
flights.select(
    delay = pl.col('dep_delay').mean()
)

delay
f64
12.705147


## grouped summaries

In [36]:
query = f'''
from flights
select year, month, day, delay: mean(dep_delay)
group by year, month, day
order by year, month, day
limit 8
'''

con = duckdb.connect('./db/nycflights13.duckdb')
result = con.sql(query).df()
con.close()

display(result)

,year,month,day,delay
0,2013,1,1,8.377778
1,2013,1,2,8.653333
2,2013,1,3,13.366667
3,2013,1,4,9.240000
4,2013,1,5,8.144928
5,2013,1,6,6.808219
6,2013,1,7,3.595745
7,2013,1,8,3.553191


In [37]:
flights.group_by(['year', 'month', 'day']).agg(
    delay = pl.col('dep_delay').mean()
).sort(['year', 'month', 'day']).head(8)

year,month,day,delay
i64,i64,i64,f64
2013,1,1,8.377778
2013,1,2,8.653333
2013,1,3,13.366667
2013,1,4,9.24
2013,1,5,8.144928
2013,1,6,6.808219
2013,1,7,3.595745
2013,1,8,3.553191


## missing values

In [ ]:
query = f'''
from flights
select
    year: count_if(year is null),
    month: count_if(month is null),
    day: count_if(day is null),
    dep_time: count_if(dep_time is null),
    dep_delay: count_if(dep_delay is null)
    -- ...
'''

con = duckdb.connect('./db/nycflights13.duckdb')
result = con.sql(query).df()
con.close()

display(result)

,year,month,day,dep_time,dep_delay
0,0.0,0.0,0.0,0.0,0.0


In [39]:
flights.select(
    pl.all().is_null().sum(),
)

year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## joins

In [65]:
display(flights.head(1))
display(airports.head(1))
display(planes.head(1))

year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
i64,i64,i64,i64,i64,i64,i64,str,str,i64,str,str,i64,i64,i64,i64
2013,6,30,940,15,1216,-4,"""VX""","""N626VA""",407,"""JFK""","""LAX""",313,2475,9,40


faa,name,lat,lon,alt,tz,dst,tzone
str,str,f64,f64,i64,i64,str,str
"""04G""","""Lansdowne Airport""",41.130472,-80.619583,1044,-5,"""A""","""America/New_York"""


tailnum,year,type,manufacturer,model,engines,seats,speed,engine
str,str,str,str,str,i64,i64,str,str
"""N10156""","""2004""","""Fixed wing multi engine""","""EMBRAER""","""EMB-145XR""",2,55,"""NA""","""Turbo-fan"""


In [67]:
query = f'''
from (
    from flights
    select
        dest,
        avg_delay: round(mean(arr_delay), 2),
    group by dest
    order by avg_delay desc) f
        left join airports a
            on f.dest = a.faa
select * exclude(a.faa, a.alt, a.tz, a.dst)
order by avg_delay desc
'''

con = duckdb.connect('./db/nycflights13.duckdb')
result = con.sql(query).df()
con.close()

display(result)

,dest,avg_delay,name,lat,lon,tzone
0,CAE,53.33,Columbia Metropolitan,33.938833,-81.119528,America/New_York
1,BHM,34.68,Birmingham Intl,33.562942,-86.753550,America/Chicago
2,OKC,30.38,Will Rogers World,35.393089,-97.600733,America/Chicago
3,TUL,29.60,Tulsa Intl,36.198389,-95.888111,America/Chicago
4,JAC,25.50,Jackson Hole Airport,43.607333,-110.737750,America/Denver
...,...,...,...,...,...,...
97,PSP,-2.00,Palm Springs Intl,33.829667,-116.506694,America/Los_Angeles
98,OAK,-2.60,Metropolitan Oakland Intl,37.721278,-122.220722,America/Los_Angeles
99,STT,-7.75,None,NaN,NaN,None
100,SNA,-8.47,John Wayne Arpt Orange Co,33.675667,-117.868222,America/Los_Angeles


In [76]:
(flights
  .group_by('dest')
  .agg(avg_delay = pl.col('arr_delay').mean().round(2))
  .join(airports, left_on='dest', right_on='faa')
  .drop('alt', 'tz', 'dst')
  .sort('avg_delay', descending=True)
)

dest,avg_delay,name,lat,lon,tzone
str,f64,str,f64,f64,str
"""CAE""",53.33,"""Columbia Metropolitan""",33.938833,-81.119528,"""America/New_York"""
"""BHM""",34.68,"""Birmingham Intl""",33.562942,-86.75355,"""America/Chicago"""
"""OKC""",30.38,"""Will Rogers World""",35.393089,-97.600733,"""America/Chicago"""
"""TUL""",29.6,"""Tulsa Intl""",36.198389,-95.888111,"""America/Chicago"""
"""JAC""",25.5,"""Jackson Hole Airport""",43.607333,-110.73775,"""America/Denver"""
…,…,…,…,…,…
"""PDX""",-1.88,"""Portland Intl""",45.588722,-122.5975,"""America/Los_Angeles"""
"""PSP""",-2.0,"""Palm Springs Intl""",33.829667,-116.506694,"""America/Los_Angeles"""
"""OAK""",-2.6,"""Metropolitan Oakland Intl""",37.721278,-122.220722,"""America/Los_Angeles"""
